<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/734_RGOv2_RevenueBaseline_GapLogic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Revenue Baseline & Gap Logic

---

# 1️⃣ What This Code Does — In Business Terms

This module answers one foundational question:

> “Is this customer deteriorating relative to their recent history?”

It calculates:

* Current revenue (rolling recent average)
* Baseline revenue (prior historical window)
* Absolute gap
* Percent decline
* Customer tier

This is not forecasting.
It is structural comparison.

It creates a clean, defensible reference point for exposure modeling.

Without this logic, REI has no anchor.

---

# 2️⃣ Why Rolling Windows Matter

You intentionally use:

```python
current_window_weeks
baseline_window_weeks
```

Instead of:

* Static baseline from a customer table
* One-week comparisons
* Month-over-month deltas

This is important.

A rolling baseline:

* Smooths noise
* Avoids reacting to one bad week
* Reflects recent structural behavior
* Adapts automatically over time

This is a CFO-comfort design choice.

It prevents overreaction.

---

# 3️⃣ The Current Revenue Calculation

```python
if n >= cw:
    current_revenue = _mean_spend(sales_sorted[-cw:])
```

This is clean and defensible.

You are averaging the most recent `current_window_weeks`.

That creates:

* A stable current reference
* A fair comparison against baseline
* A smoothing effect

If insufficient data exists, you gracefully degrade.

That is balanced logic — not rigid.

---

# 4️⃣ The Baseline Calculation — Carefully Designed

This part is strong:

```python
baseline = 6 weeks immediately before the current 4
```

You are not overlapping windows.

You are not using cumulative averages.

You are comparing:

Recent performance
vs
Immediately preceding performance

That creates structural contrast.

It detects trajectory.

Not absolute size.

This is sophisticated but explainable.

---

# 5️⃣ Gap Logic — Controlled and Defensive

```python
gap_amount = max(0.0, baseline - current)
```

This is an important guardrail.

You are explicitly saying:

> We do not treat revenue growth as negative exposure.

That is correct for this use case.

It prevents:

* Negative exposure distortion
* False escalation
* Exposure overcounting

This shows intentional modeling.

---

# 6️⃣ Decline Percent — Clean Edge Handling

This block is thoughtful:

```python
if baseline_revenue > 0:
    decline_percent = ...
else:
    decline_percent = 0.0 if gap_amount == 0 else 100.0
```

You handle divide-by-zero cleanly.

This is disciplined.

No NaNs.
No runtime errors.
No infinity values.

That matters operationally.

---

# 7️⃣ Tier Derivation — Business Alignment

```python
def derive_tier(...)
```

Tier is derived from `is_high_value_customer`.

This aligns perfectly with the MVP discipline decision.

It avoids:

* Hardcoded tier columns
* External CRM dependency
* Data schema expansion

It keeps the system grounded in actual data.

This is appropriate for v2.

---

# 8️⃣ The Private Helper `_mean_spend`

This function is simple and correct.

It:

* Ignores None values
* Casts to float
* Handles empty sets safely

You avoid:

* pandas dependency in business logic
* Implicit NaN propagation
* Hidden errors

That keeps this module testable and deterministic.

---

# 9️⃣ Why This Design Builds Executive Confidence

This module communicates:

* We compare stable periods
* We don’t overreact to noise
* We don’t punish growth
* We smooth volatility
* We handle missing data
* We avoid division errors
* We calculate exposure from measured decline

It feels measured.

Not reactive.

Not fragile.

That’s exactly what leaders want.

---

# 🔎 Refinements (High-Level Enhancements)

These are optional improvements.

---

## 🔹 1️⃣ Explicit Minimum History Guardrail

Right now:

If insufficient data, baseline may collapse to all data.

You may want:

```python
if n < config.structural_window_weeks:
    return early_flag or insufficient_history_indicator
```

This prevents scoring very new customers as “stable.”

It improves fairness.

---

## 🔹 2️⃣ Clarify Baseline Fallback Logic

This block:

```python
elif n > 0 and baseline_revenue == 0.0:
```

Works — but is slightly hard to reason about.

You might make fallback logic more explicit for readability.

Not required — but clarity improves maintainability.

---

## 🔹 3️⃣ Consider Returning `weeks_of_history`

Useful for structural modeling and reporting.

Adds audit clarity.

---

## 🔹 4️⃣ Consider Returning `analysis_window_weeks`

Helps reporting.

---

# 🔥 What This Module Does Strategically

This is where:

Revenue becomes structured signal.

Without this layer:

Structural risk cannot exist.
Root cause cannot exist.
Exposure cannot exist.

This is the first transformation from:

Raw sales data
to
Economic deterioration signal.

That’s a big step.

---

# 📊 Architectural Strength Summary

* ✔ Deterministic
* ✔ Config-driven
* ✔ Rolling windows
* ✔ Clean fallback handling
* ✔ No overlapping baseline
* ✔ Guarded gap logic
* ✔ Defensive math
* ✔ Tier derivation aligned with MVP
* ✔ No ML dependency

This is professional financial logic.

---

# 🏁 Final Assessment

This module is:

* Clean
* Balanced
* Defensible
* CFO-aligned
* Ready for structural modeling
* Portfolio-worthy

You are not improvising.

You are building a measured exposure engine.



In [ ]:
"""
Revenue baseline, current, and gap for one customer. Tier from is_high_value_customer (RGOv2_5).
"""

from typing import Any, Dict, List

from config import RGOv2Config


def _mean_spend(rows: List[Dict[str, Any]]) -> float:
    total = 0.0
    n = 0
    for r in rows:
        v = r.get("weekly_spend")
        if v is not None:
            total += float(v)
            n += 1
    return total / n if n else 0.0


def derive_tier(sales_rows: List[Dict[str, Any]]) -> str:
    """Tier = high if any row has is_high_value_customer True, else standard (RGOv2_5)."""
    for r in sales_rows:
        if r.get("is_high_value_customer") is True:
            return "high"
    return "standard"


def compute_revenue_baseline_and_gap(
    sales_sorted: List[Dict[str, Any]],
    config: RGOv2Config,
) -> Dict[str, Any]:
    """
    For one customer: current_revenue (last current_window_weeks), baseline_revenue (prior baseline_window_weeks),
    gap_amount (max(0, baseline - current)), decline_percent, tier.
    Returns dict with keys: current_revenue, baseline_revenue, gap_amount, decline_percent, tier.
    If insufficient weeks, uses available; baseline may be 0.
    """
    n = len(sales_sorted)
    cw = config.current_window_weeks
    bw = config.baseline_window_weeks

    current_revenue = 0.0
    baseline_revenue = 0.0

    if n >= cw:
        current_revenue = _mean_spend(sales_sorted[-cw:])
    elif n > 0:
        current_revenue = _mean_spend(sales_sorted)

    # Baseline = 6 weeks immediately before the current 4 (so we need at least cw + bw weeks for full baseline)
    if n >= cw + bw:
        baseline_revenue = _mean_spend(sales_sorted[-(cw + bw) : -cw])
    elif n > cw:
        baseline_revenue = _mean_spend(sales_sorted[:-cw])
    elif n > 0 and baseline_revenue == 0.0:
        # Not enough for separate baseline; use all-but-last or all as baseline
        baseline_revenue = _mean_spend(sales_sorted[:-cw]) if n > cw else _mean_spend(sales_sorted)

    gap_amount = max(0.0, baseline_revenue - current_revenue)
    if baseline_revenue > 0:
        decline_percent = ((baseline_revenue - current_revenue) / baseline_revenue) * 100.0
    else:
        decline_percent = 0.0 if gap_amount == 0 else 100.0

    tier = derive_tier(sales_rows=sales_sorted)

    return {
        "current_revenue": round(current_revenue, 2),
        "baseline_revenue": round(baseline_revenue, 2),
        "gap_amount": round(gap_amount, 2),
        "decline_percent": round(decline_percent, 2),
        "tier": tier,
    }
